# Observability를 갖춘 Amazon Bedrock AgentCore Runtime에서 Amazon Bedrock model을 사용하는 LlamaIndex Agent 호스팅

## 개요

이 튜토리얼에서는 내장된 observability를 갖춘 Amazon Bedrock AgentCore Runtime을 사용하여 LlamaIndex agent를 호스팅하는 방법을 배웁니다. 이 튜토리얼은 LlamaIndex agent를 AgentCore Runtime에 배포하고 모니터링 및 분석을 위한 텔레메트리 데이터를 자동으로 캡처하는 방법을 보여줍니다.

### 튜토리얼 세부 정보

| 정보                | 세부 사항                                                                          |
|:--------------------|:---------------------------------------------------------------------------------|
| 튜토리얼 유형       | 대화형                                                                            |
| Agent 유형          | 단일                                                                              |
| Agentic Framework   | LlamaIndex                                                                       |
| LLM model           | Anthropic Claude Haiku                                                           |
| 튜토리얼 구성 요소  | Observability를 갖춘 AgentCore Runtime에서 agent 호스팅                          |
| 튜토리얼 분야       | 범용                                                                              |
| 예제 복잡도         | 쉬움                                                                              |
| 사용된 SDK          | Amazon BedrockAgentCore Python SDK 및 boto3                                     |

### 튜토리얼 아키텍처

이 튜토리얼에서는 자동 observability를 갖춘 AgentCore runtime에 LlamaIndex agent를 배포하는 방법을 설명합니다.

데모 목적으로 산술 도구를 사용하는 Amazon Bedrock model을 활용한 LlamaIndex FunctionAgent를 사용합니다.

### 튜토리얼 주요 기능

* Amazon Bedrock AgentCore Runtime에서 LlamaIndex Agent 호스팅
* Amazon Bedrock model 사용
* 자동 observability 및 추적
* 내장 텔레메트리 수집

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다:
* Python 3.10+
* AWS 자격 증명
* Amazon Bedrock AgentCore SDK
* LlamaIndex
* Claude Haiku에 대한 Amazon Bedrock Model 액세스

### 터미널에서:

`cd 01-tutorials/06-AgentCore-observability/01-Agentcore-runtime-hosted/LlamaIndex`

`python -m venv venv`

`source venv/bin/activate`

### Notebook에서:

venv를 커널로 선택하세요

In [ ]:
!pip install -q --force-reinstall -U -r requirements.txt

## LlamaIndex agent 생성 및 로컬에서 실험하기

agent를 AgentCore Runtime에 배포하기 전에 실험 목적으로 로컬에서 개발하고 실행해 보겠습니다.

프로덕션 agentic 애플리케이션의 경우 agent 생성 프로세스와 agent 호출 프로세스를 분리해야 합니다. AgentCore Runtime을 사용하면 agent의 호출 부분을 `@app.entrypoint` 데코레이터로 장식하고 이를 runtime의 진입점으로 사용합니다.

In [ ]:
%%writefile llamaindex_agent.py

import warnings
# validate_default 관련 경고 메시지 무시
warnings.filterwarnings("ignore", message=".*validate_default.*", category=UserWarning)
import os
import json
import argparse
import boto3
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.observability.otel import LlamaIndexOpenTelemetry


# LlamaIndex용 OpenTelemetry instrumentation 초기화
instrumentor = LlamaIndexOpenTelemetry()
# 텔레메트리 수집 시작
instrumentor.start_registering()

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def get_bedrock_model():
    model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
    # 현재 boto3 세션의 region 사용
    region = boto3.Session().region_name
    
    bedrock_model = BedrockConverse(
        model=model_id,
        region_name=region,
    )
    return bedrock_model

# Bedrock model 초기화
bedrock_model = get_bedrock_model()

# 산술 연산 도구를 가진 agent 생성
agent = FunctionAgent(
    tools=[add, multiply],
    llm=bedrock_model,
)

async def llamaindex_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    # agent 실행 (비동기)
    response = await agent.run(user_input)
    return str(response)

if __name__ == "__main__":
    import asyncio
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    # JSON payload를 파싱하여 agent 실행
    response = asyncio.run(llamaindex_agent_bedrock(json.loads(args.payload)))
    print(response)

#### 로컬 agent 호출

In [ ]:
!python llamaindex_agent.py '{"prompt": "What is (121 + 2) * 5?"}'

## AgentCore Runtime에 배포하기 위한 agent 준비

이제 agent를 AgentCore Runtime에 배포해 보겠습니다. 이를 위해 다음을 수행해야 합니다:
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp`으로 Runtime App 가져오기
* `app = BedrockAgentCoreApp()`으로 코드에서 App 초기화
* `@app.entrypoint` 데코레이터로 호출 함수 장식
* `app.run()`으로 AgentCoreRuntime이 agent 실행을 제어하도록 설정

### Amazon Bedrock model을 사용하는 LlamaIndex Agent
AgentCore Runtime 배포를 위해 LlamaIndex Agent를 준비해 보겠습니다.

In [ ]:
%%writefile llamaindex_agent.py

import warnings
# validate_default 관련 경고 메시지 무시
warnings.filterwarnings("ignore", message=".*validate_default.*", category=UserWarning)
import os
import json
import boto3
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from llama_index.llms.bedrock_converse import BedrockConverse
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.observability.otel import LlamaIndexOpenTelemetry


# AgentCore Runtime App 초기화
app = BedrockAgentCoreApp()

# LlamaIndex용 OpenTelemetry instrumentation 초기화 (debug 모드)
instrumentor = LlamaIndexOpenTelemetry(debug=True)
# 텔레메트리 수집 시작
instrumentor.start_registering()

def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

def get_bedrock_model():
    model_id = "anthropic.claude-3-5-haiku-20241022-v1:0"
    # 현재 boto3 세션의 region 사용
    region = boto3.Session().region_name
    
    bedrock_model = BedrockConverse(
        model=model_id,
        region_name=region,
    )
    return bedrock_model

# Bedrock model 초기화
bedrock_model = get_bedrock_model()

# 산술 연산 도구를 가진 agent 생성
agent = FunctionAgent(
    tools=[add, multiply],
    llm=bedrock_model,
)

# AgentCore Runtime의 진입점으로 지정
@app.entrypoint
async def llamaindex_agent_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    # agent 실행 (비동기)
    response = await agent.run(user_input)
    return str(response)

if __name__ == "__main__":
    # AgentCore Runtime 실행
    app.run()

## 내부에서 무슨 일이 일어나나요?

`BedrockAgentCoreApp`을 사용하면 자동으로:

* 포트 8080에서 수신 대기하는 HTTP 서버 생성
* agent의 요구 사항을 처리하기 위한 필수 `/invocations` 엔드포인트 구현
* 상태 확인을 위한 `/ping` 엔드포인트 구현
* 적절한 콘텐츠 유형 및 응답 형식 처리
* AWS 표준에 따른 오류 처리 관리
* **observability 및 텔레메트리 수집 자동 활성화**

## AgentCore Runtime에 agent 배포

`CreateAgentRuntime` 작업은 포괄적인 구성 옵션을 지원하여 컨테이너 이미지, 환경 변수 및 암호화 설정을 지정할 수 있습니다. 또한 프로토콜 설정(HTTP, MCP) 및 권한 부여 메커니즘을 구성하여 클라이언트가 agent와 통신하는 방식을 제어할 수 있습니다.

**참고:** 운영 모범 사례는 코드를 컨테이너로 패키징하고 CI/CD 파이프라인 및 IaC를 사용하여 ECR에 푸시하는 것입니다

이 튜토리얼에서는 Amazon Bedrock AgentCore Python SDK를 사용하여 아티팩트를 쉽게 패키징하고 AgentCore runtime에 배포합니다.

### AgentCore Runtime 배포 구성

먼저 스타터 툴킷을 사용하여 진입점, 방금 생성한 실행 역할 및 requirements 파일로 AgentCore Runtime 배포를 구성합니다. 또한 시작 시 Amazon ECR 리포지토리를 자동으로 생성하도록 스타터 킷을 구성합니다.

구성 단계에서 애플리케이션 코드를 기반으로 docker 파일이 생성됩니다

In [ ]:
from bedrock_agentcore_starter_toolkit.notebook.runtime.bedrock_agentcore import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name

agentcore_runtime = Runtime()
agent_name = "llamaindex_bedrock_getting_started10"
# AgentCore Runtime 배포 구성
response = agentcore_runtime.configure(
    entrypoint="llamaindex_agent.py",  # agent 진입점 파일
    auto_create_execution_role=True,  # 실행 역할 자동 생성
    auto_create_ecr=True,  # ECR 리포지토리 자동 생성
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)
response

### AgentCore Runtime에 agent 시작

이제 docker 파일이 준비되었으므로 agent를 AgentCore Runtime에 시작해 보겠습니다. 이렇게 하면 Amazon ECR 리포지토리와 observability가 자동으로 활성화된 AgentCore Runtime이 생성됩니다. 아래 환경 변수에 라이브러리를 추가하여 특정 라이브러리에서 불필요한 추적을 제외할 수 있습니다.

In [ ]:
# AgentCore Runtime에 agent 배포 및 시작
launch_result = agentcore_runtime.launch(
    env_vars={
        # 불필요한 instrumentation 비활성화하여 노이즈 감소
        "OTEL_PYTHON_DISABLED_INSTRUMENTATIONS": (
            "jinja2,"
            "urllib3,"
            "requests,"
            "httpx,"
            "redis,"
            "aiohttp-client"
            # "starlette"를 추가하면 POST /invocations trace 제거 가능 (단, 세션 추적 비활성화됨)
        )
    }
)


### AgentCore Runtime 상태 확인
이제 AgentCore Runtime을 배포했으므로 배포 상태를 확인해 보겠습니다

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
# Runtime이 준비될 때까지 폴링
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

## AgentCore Runtime에 대한 추적 활성화

AWS 콘솔에서 Amazon Bedrock AgentCore로 이동합니다. Build and Deploy 아래의 Agent Runtime을 클릭하고 agent를 선택합니다.

agent에 대해 Tracing 섹션이 보일 때까지 스크롤합니다. CloudWatch로 추적 전달을 허용하려면 이를 활성화하세요:

![enable_tracing.png](images/llamaindex_enable_tracing.png)

### AgentCore Runtime 호출

마지막으로 페이로드로 AgentCore Runtime을 호출할 수 있습니다. 이렇게 하면 observability 대시보드에서 볼 수 있는 텔레메트리 데이터가 자동으로 생성됩니다.

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "What is (121 + 2) * 5?"})
invoke_response

### 호출 결과 처리

이제 호출 결과를 처리하여 애플리케이션에 포함할 수 있습니다

In [ ]:
from IPython.display import Markdown, display
import json
# 응답에서 텍스트 추출
response_text = invoke_response['response'][0]
display(Markdown(response_text))

### boto3로 AgentCore Runtime 호출

이제 AgentCore Runtime이 생성되었으므로 모든 AWS SDK로 호출할 수 있습니다. 예를 들어 boto3 `invoke_agent_runtime` 메서드를 사용할 수 있습니다.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
# Bedrock AgentCore 클라이언트 생성
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

# boto3로 agent runtime 호출
boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 15 * 8?"})
)
# 응답 타입에 따라 처리
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    # EventStream 응답 처리
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                print(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Observability 대시보드

### 자동 텔레메트리 수집

LlamaIndex agent가 AgentCore Runtime에서 실행되면 텔레메트리 데이터가 자동으로 수집되어 Amazon CloudWatch로 전송됩니다. 여기에는 다음이 포함됩니다:

- **Agent 실행 추적**: agent의 의사 결정 프로세스의 완전한 워크플로우
- **LLM 호출**: 입력/출력 토큰이 포함된 Bedrock model 호출
- **Tool 사용**: 함수 호출 및 결과
- **성능 메트릭**: 지연 시간, 토큰 사용량 및 오류율

### CloudWatch에서 추적 보기

agent의 observability 데이터를 보려면:

1. AWS CloudWatch 콘솔로 이동
2. **GenAI Observability** 대시보드로 이동
3. agent runtime을 선택하여 추적 및 메트릭 보기

### 주요 Observability 기능

- **세션 추적**: 여러 상호 작용 연관
- **오류 모니터링**: 문제 식별 및 디버그
- **성능 분석**: agent 응답 시간 최적화

### Amazon CloudWatch의 AgentCore Observability

요약하자면, AgentCore runtime에서 호스팅되는 agent의 observability를 활성화하려면 다음 단계를 따르세요:

- Amazon CloudWatch에서 Transaction Search 활성화
- Bedrock AgentCore Runtime에 agent를 배포할 때 requirements.txt 파일에 `aws-opentelemetry-distro`가 나열되어 있어야 합니다.

## GenAI Observability 대시보드의 Bedrock AgentCore 개요

observability가 있는 모든 Agent를 볼 수 있으며 시간 프레임을 기준으로 데이터를 필터링할 수 있습니다. 몇 가지 예는 다음과 같습니다:

![genai-observability.png](images/llamaindex_dashboard_view.png)

메인 대시보드에서는 아래와 같이 모든 agent의 runtime 메트릭을 볼 수 있습니다:

![runtime-all-agent-metrics.png](images/llamaindex_runtime_metrics_total_view.png)

이제 방금 배포한 agent를 클릭하면 이 agent에 특정한 runtime 메트릭 대시보드로 이동하며, 사용자 지정 시간 프레임으로 데이터를 필터링할 수도 있습니다:

![runtime-metrics-per-agent.png](images/llamaindex_runtime_metrics_view.png)

Sessions View 탭에서 이 agent와 관련된 모든 세션으로 이동할 수 있습니다:

![Agent-sessions-view.png](images/llamaindex_sessions_view.png)

Trace View 탭에서 runtime의 이 agent에 대한 추적 및 span 정보를 확인할 수 있습니다:

![Agentcore-trace.png](images/llamaindex_traces_view.png)

추적에 대한 자세한 정보를 얻으려면 GenAI observability 대시보드의 다양한 기능을 클릭하세요.


## 정리 (선택 사항)

이제 생성된 AgentCore Runtime을 정리해 보겠습니다

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
import boto3
# Bedrock AgentCore Control 클라이언트 생성
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
# ECR 클라이언트 생성
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

# Agent Runtime 삭제
runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

# ECR 리포지토리 삭제 (force=True로 이미지 포함 삭제)
response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

# 축하합니다!

다음을 성공적으로 완료했습니다:

- 산술 도구를 사용하는 LlamaIndex agent 생성
- Amazon Bedrock AgentCore Runtime에 배포
- 자동 observability 및 텔레메트리 수집 활성화
- agent 호출 및 추적 데이터 생성

이제 agent가 완전한 observability 기능으로 실행되어 성능을 모니터링하고, 문제를 디버그하고, agentic 애플리케이션을 최적화할 수 있습니다.